# WHAT WE WILL DO IN THIS NOTEBOOK
- Data cleaning
- Basic EDA
- Visualization
- Background knowledge about startup financing.

## This notebook is stil under construction.
## Feel free to <font color=deepskyblue> FORK  </font> this notebook, Please  <font color=deepskyblue> UPVOTE !! </font> if it's helpful to you  <font color=deepskyblue> : ) </font>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
import seaborn as sns

# 1.Data Cleaning
Data cleaning is the single most important and time-consuming part of data analysis.you may need to come back to it during the whole analysis process now and then.

**What We Know:**
1. Encryption of this data set is not normal when you try to **pd.read_csv()**
2. We have 54294 entries, 39 columns.
3. There are a lot of duplicated entries which are actually Nans
4. Column names `market, funding_total_usd` have spaces,we gonna fix that
**What we gonna do:**
1. We use **chardet.detect()** to detect encryption of this data set.
2. We use **.duplicated()** and **.dropna()** to find and drop those Nan entries.
3. We use **.rename()** to change column names

In [ ]:
path='/kaggle/input/startup-investments-crunchbase/investments_VC.csv'
f = open(path,'rb')
line = f.readline()
print(chardet.detect(line))

In [ ]:
data=pd.read_csv(path,encoding='unicode_escape')

In [ ]:
data.info()

In [ ]:
data.rename(columns={' market ':'market',' funding_total_usd ':'funding_total_usd'},inplace=True)

In [ ]:
data.head(10)

## Checking Duplicated Entries

**What we do**

We have a lot of duplicated entries,most of them are Nans, we gonna get ride of them.

In [ ]:
print('Duplicated entries:',data.duplicated().sum())
data[data.duplicated()].isna().mean()

In [ ]:
data.dropna(how='all',inplace=True)

## Checking Nans

Normally, you need to check every columns carefully to ensure the validation of our data.

For Nans,we have 3 ways to deal with:
1. try to find why they are missing.
2. Replace or drop them
3. Pick them out for later research.

But we just gonna leave them there this time, for it's enough for our analysis.

In [ ]:
data_na=data.isna().mean().to_frame()
data_na[data_na[0]>0].style.background_gradient(cmap='Blues')

# 2. Quick analysis

**What we know:**
1. We have startups established from **1902 to 2014**. 

   *yes, The Miriam Hospital is actually founded in 1902 and opened in 1926, try using `data[data.founded_year==1902]` to find it*
2. Numbers of startups are strongly left skewed. 3 contries that have most startups in our dataset is US,GBR and CAN
3. 86% of our startups are still operating at the time this data is collected. Remember this could change during the time.
4. If you are starting a startup in CHN (China), you got more chance to operating than acquired or closed.

In [ ]:
data.founded_year.describe()

In [ ]:
plt.style.use('fivethirtyeight')
sns.distplot(data.country_code.value_counts(),kde=False)
plt.axvline(data.loc[data.country_code=='USA'].shape[0],color='black',linestyle ="--",linewidth=1)
plt.text(data.loc[data.country_code=='USA'].shape[0]-5000, 80,"   USA \n28793")
plt.axvline(data.loc[data.country_code=='GBR'].shape[0],color='coral',linestyle ="--",linewidth=1)
plt.text(data.loc[data.country_code=='GBR'].shape[0]+100, 70,"GBR \n2642",color='coral')
plt.axvline(data.loc[data.country_code=='CAN'].shape[0],color='limegreen',linestyle ="--",linewidth=1)
plt.text(data.loc[data.country_code=='CAN'].shape[0]+100, 50,"CAN \n2642",color='limegreen')
plt.title('Distribution of start-ups\nAcross countries',loc='left',pad=10)
plt.xlabel('Quantity of Start-ups')
plt.show()

In [ ]:
import numpy as np
status=data.status.value_counts()
plt.figure()

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
plt.pie(status.values, radius=2, colors=outer_colors,labels=status.index,autopct='%1.1f%%',
       wedgeprops=dict(width=0.5, edgecolor='w'))
plt.title('Status of Start-Ups',pad=100)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
status_cont=data.groupby('country_code')['status'].value_counts(normalize=True)
cont=['USA','GBR','CAN','CHN']
status_cont=status_cont.loc[cont].unstack().T
plt.bar(status_cont.columns,status_cont.loc['acquired'],color=outer_colors[1],label='acquired',width=0.5,edgecolor='w')
plt.bar(status_cont.columns,status_cont.loc['closed'],bottom=status_cont.loc['acquired'],color=outer_colors[2],label='closed',width=0.5,edgecolor='w')
plt.bar(status_cont.columns,status_cont.loc['operating'],bottom=status_cont.loc['closed']+status_cont.loc['acquired'],color=outer_colors[0],label='operating',width=0.5,edgecolor='w')
plt.legend()
plt.title('Status of Start-ups in 4 Contries')
plt.show()

# 3. Markets and Status

**What we know:**
1. Top 10 markets of all startups are *software, biotech, Mobile. E-commerce etc*
2. Top 10 markets of USA, CAN, GBR are basically the same.
3. The first one is *E-commerce*,and no *clean tech* on that list in CHINA.
4. *Public Relations, coupons* have the highest closed rate, while *Consumer Electronics, Mediacal* have the lowest closed rate.
        *technology* is a vague words.
5. *Chat, cloud management* have the highest rate to be acquired.

In [ ]:
market=data.market.value_counts()
market[(market>1000)][0:10]
market=market[(market>1000)][0:10].to_frame()

In [ ]:
plt.figure()
ax1=sns.barplot(y=market.index,x=market.market,orient='h',palette='Blues_r')
ax1.set_title('Top 10 Market Counts of Start-Ups',pad=20)
ax1.set_xlabel('Market Counts Global')
plt.show()

In [ ]:
market_con=data.groupby('country_code')['market'].value_counts().to_frame()

In [ ]:
fig,axes=plt.subplots(2,2,figsize=(20,10))
cont=['USA','GBR','CAN','CHN']
for con,ax in zip(cont,range(1,5)):
    ax=plt.subplot(2,2,ax)
    ax=sns.barplot(y=market_con.loc[con].index[0:10],x=market_con.loc[con].market[0:10],orient='h',palette='Blues_r')
    ax.set_xlabel('Market Counts In {}'.format(con))
plt.suptitle('Top 10 Market of Start-ups',size=40)
plt.show()

In [ ]:
top_markets=(data.market.value_counts()[data.market.value_counts()>24]).index
# We chose top markets to analyze,which are greater than 24 startups in that market.
sub_data=data[data.market.isin(top_markets)]

status_market_1=sub_data.groupby('market')['status'].value_counts(normalize=True).to_frame()
status_market_1.columns=['prop']
status_market_2=sub_data.groupby('market')['status'].value_counts().to_frame()
status_market_2.columns=['quantity']
status_market=pd.concat([status_market_1,status_market_2],axis=1)
status_market.reset_index()
status_market_closed=status_market.query('status=="closed"').reset_index(level=1)
status_market_closed.sort_values(by='prop',ascending=False)[0:10].style.background_gradient(cmap='Reds',subset=['prop']).set_caption('Markets that most start-ups closed')

In [ ]:
status_market_closed.sort_values(by='prop',ascending=True)[0:10].style.background_gradient(cmap='Greens_r',subset=['prop']).set_caption('Markets that least start-ups closed')

In [ ]:
status_market_acquired=status_market.query('status=="acquired"').reset_index(level=1)
status_market_acquired.sort_values(by='prop',ascending=False)[0:10].style.background_gradient(cmap='Blues',subset=['prop']).set_caption('Markets that most start-ups acquired')

# 4. Fundings
**What we know:**
1. The funding distribution are strongly left skewed.
2. Most startups' total funding is less than 0.5bn
3. We got 54294 companies here, top 10% take 75.5% of total money.
4. Verizon is a giant one.
5. Funding rounds have relatively strong relation to startups's status, acquired ones are tend to get more rounds of fundings.
6. Most company only get one round of funding.

In [ ]:
data.funding_total_usd=data.funding_total_usd.str.replace(",",'').str.replace(' ',"")
data.funding_total_usd=data.funding_total_usd.replace("-",np.nan).astype('float',errors='ignore')
fig=plt.figure(figsize=(15,5))
sns.distplot(data[~data.funding_total_usd.isna()]['funding_total_usd'],kde=False,hist_kws={'log':True})
plt.show()

### We got 54294 companies here, top 10% take 75.5% of total money.

It's a winner takes all game.

In [ ]:
top_fund=(data.funding_total_usd.sort_values(ascending=False)[1:5000].sum())/(data.funding_total_usd.sort_values(ascending=False).sum())

plt.figure()
cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(2)*13)
plt.pie([top_fund,1-top_fund], radius=2, colors=outer_colors,labels=['Top 10%','Others'],autopct='%1.1f%%',
       wedgeprops=dict(width=0.5, edgecolor='w'))
plt.title('Top 10% Companies\' total funding',pad=100)
plt.show()

In [ ]:
data.sort_values(ascending=False,by='funding_total_usd')[['name','funding_total_usd']][0:10]\
.style.hide_index().background_gradient(subset = 'funding_total_usd',cmap='Blues')

In [ ]:
data1=data.sort_values(ascending=False,by='funding_total_usd')[['name','funding_total_usd']][1:5000].sort_index()
plt.figure(figsize=(12,8))
sns.scatterplot(data=data1,x='name',y='funding_total_usd',size='funding_total_usd')
plt.xticks([])
plt.title('Funding total in USD')
plt.xlabel("")
plt.show()

In [ ]:
founded_year=data[~data.founded_year.isna()]['founded_year'].astype('int')

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(founded_year[founded_year>1980])
plt.title('Founded year distribution')
plt.show()

In [ ]:
%%time
category_list=data.category_list.str.split('|').to_list()
categ=[]

for i in category_list:
    if isinstance(i,list):
        categ=categ+i
categ=list(filter(None,categ))
categ=' '.join(categ)

In [ ]:
import wordcloud
w=wordcloud.WordCloud(background_color='White',scale=32)

In [ ]:
fig = plt.figure(figsize=(12, 8))
w.generate(categ)
plt.axis('off')
plt.imshow(w)

In [ ]:
sns.boxplot(data=data,x='status',y='funding_rounds')
data.groupby('status')['funding_rounds'].describe().style.highlight_max(subset=['max','75%','50%','std','mean'],color='lightgreen')

In [ ]:
funding_rounds=data.funding_rounds.value_counts()
other=funding_rounds[funding_rounds.index>=7].sum()
funding_rounds=funding_rounds[funding_rounds.index<7]
funding_rounds.index=funding_rounds.index.astype('int').astype('str')
funding_rounds.loc['more']=other

In [ ]:
sns.barplot(x=funding_rounds.index,y=funding_rounds.values,palette='Paired')
plt.show()